# Пример общего рабочего процесса обработки осциллограмм

Этот ноутбук демонстрирует возможную последовательность шагов при обработке коллекции осциллограмм с использованием ранее созданных модульных классов. Каждый этап может быть адаптирован, пропущен или выполнен независимо в соответствии с конкретными задачами.

**Этапы рабочего процесса:**
1.  **Обнаружение и сбор осциллограмм**: Используем `OscillogramFinder` для поиска и копирования файлов нужных типов из исходной директории (включая архивы) в рабочую директорию.
2.  **Анонимизация**: Применяем `DataAnonymizer` для удаления конфиденциальной информации и переименования файлов по хеш-сумме.
3.  **Каталогизация имен сигналов**: Используем `SignalNameManager` для создания каталога всех имен сигналов. (Опционально: их последующее переименование).
4.  **Фильтрация по активности**: Применяем `OscillogramActivityFilter` для отсеивания "пустых" или неактивных осциллограмм.
5.  **Генерация коэффициентов нормализации**: На основе активных осциллограмм создаем файл `norm.csv` с помощью `NormalizationCoefficientGenerator`.
6.  **Преобразование в CSV**: Конвертируем обработанные (анонимизированные, активные, нормализованные) осциллограммы в формат CSV с помощью `OscillogramToCsvConverter` для дальнейшего анализа или использования в ML.
7.  **Специализированный анализ (пример)**: Краткая демонстрация запуска одного из анализаторов, например, `SPEFAnalyzer` или `OvervoltageDetector`.

Все пути к файлам и директориям, а также конфигурации, являются примерами и должны быть адаптированы под реальные задачи.

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import json
import csv
import zipfile 
import sys

# Настройка путей для импорта модулей проекта
module_path = os.path.abspath(os.path.join(os.getcwd(), '..')) 
if module_path not in sys.path:
    sys.path.append(module_path)

from core.oscillogram import Oscillogram
from filesystem.finder import OscillogramFinder, TYPE_OSC
from preprocessing.anonymizer import DataAnonymizer
from preprocessing.signal_names import SignalNameManager
from analysis.activity_filter import OscillogramActivityFilter, ChannelType 
from normalization.normalization import NormalizationCoefficientGenerator, OscillogramNormalizer
from raw_to_csv.raw_to_csv import OscillogramToCsvConverter 
from analysis.spef import SPEFAnalyzer 
from analysis.overvoltage_detector import OvervoltageDetector 

base_workflow_dir = "temp_workflow_demonstration"
initial_source_dir = os.path.join(base_workflow_dir, "00_initial_source")
staging_dir_found = os.path.join(base_workflow_dir, "01_staging_found_oscillograms")
staging_dir_anonymized = os.path.join(base_workflow_dir, "02_staging_anonymized")
staging_dir_active = os.path.join(base_workflow_dir, "03_staging_active_oscillograms")
output_configs_dir = os.path.join(base_workflow_dir, "output_and_configs")

if os.path.exists(base_workflow_dir):
    shutil.rmtree(base_workflow_dir)
os.makedirs(initial_source_dir)
os.makedirs(staging_dir_found)
os.makedirs(staging_dir_anonymized)
os.makedirs(staging_dir_active)
os.makedirs(output_configs_dir)

print(f"Созданы базовые директории в: {base_workflow_dir}")

def create_workflow_cfg_dat(path, name, analog_channels_data, digital_channels_data=[], freq=50.0, samp_rate=1000.0, dat_rows=50):
    os.makedirs(path, exist_ok=True)
    cfg_content = f"InitialStation,Device_{name},1999\n"
    cfg_content += f"{len(analog_channels_data) + len(digital_channels_data)},{len(analog_channels_data)}A,{len(digital_channels_data)}D\n"
    ch_idx = 1
    for ch_name_cfg, _ in analog_channels_data:
        cfg_content += f"{ch_idx},{ch_name_cfg},A,,V,1.0,0.0,0,-32767,32767,1,1,P\n" 
        ch_idx += 1
    for ch_name_cfg, _ in digital_channels_data:
        cfg_content += f"{ch_idx},{ch_name_cfg},,,1\n"
        ch_idx += 1
    cfg_content += f"{freq}\n1\n{samp_rate},{dat_rows}\n"
    cfg_content += f"01/01/2023,00:00:00.000000\n01/01/2023,00:00:00.000000\nASCII\n1.0\n"
    with open(os.path.join(path, f"{name}.cfg"), "w", encoding="utf-8") as f: f.write(cfg_content)

    dat_content = ""
    for i in range(dat_rows):
        dat_content += f"{i+1},{int(i*(1000000/samp_rate))}"
        for _, data_series in analog_channels_data: dat_content += f",{data_series[i % len(data_series)]:.6f}"
        for _, data_series in digital_channels_data: dat_content += f",{data_series[i % len(data_series)]}"
        dat_content += "\n"
    with open(os.path.join(path, f"{name}.dat"), "w", encoding="utf-8") as f: f.write(dat_content)

time_wf = np.linspace(0, 2*np.pi*2.5, 50)
active_data = np.concatenate([np.sin(time_wf[:25])*0.5, np.sin(time_wf[25:])*1.5])
create_workflow_cfg_dat(initial_source_dir, "wf_active_osc", 
                        [("U_Active", active_data)], 
                        [("ML_Event_Dummy", np.concatenate([np.zeros(20, dtype=int),np.ones(10, dtype=int),np.zeros(20, dtype=int)]))])

stable_data = np.sin(time_wf)*0.8
create_workflow_cfg_dat(initial_source_dir, "wf_empty_osc", [("I_Stable", stable_data)])

os.makedirs(os.path.join(initial_source_dir, "subdir_initial"), exist_ok=True)
create_workflow_cfg_dat(os.path.join(initial_source_dir, "subdir_initial"), "wf_active_osc_sub", 
                        [("U_Active_Sub", active_data*0.7)])

archive_path = os.path.join(initial_source_dir, "archive_with_osc.zip")
temp_archive_files_path = os.path.join(base_workflow_dir, "temp_for_zip")
os.makedirs(temp_archive_files_path, exist_ok=True)
create_workflow_cfg_dat(temp_archive_files_path, "temp_archived_osc", [("U_Archived", stable_data*0.3)])
with zipfile.ZipFile(archive_path, 'w') as zf:
    zf.write(os.path.join(temp_archive_files_path, "temp_archived_osc.cfg"), "temp_archived_osc.cfg")
    zf.write(os.path.join(temp_archive_files_path, "temp_archived_osc.dat"), "temp_archived_osc.dat")
shutil.rmtree(temp_archive_files_path)
    
def create_dummy_file(filepath, content="dummy_content"):
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, "w", encoding="utf-8") as f: f.write(content)
create_dummy_file(os.path.join(initial_source_dir, "some_other_file.txt"), "some text")

print(f"Содержимое initial_source_dir создано.")

## Этап 1: Обнаружение и сбор осциллограмм (`OscillogramFinder`)
Копируем только файлы COMTRADE (.cfg/.dat) из `initial_source_dir` (включая архивы) в `staging_dir_found`.
Сохраняем структуру директорий и используем хеши для избежания дубликатов.

In [ ]:
finder_wf = OscillogramFinder(is_print_message=True)
initial_hashes_path_wf = os.path.join(staging_dir_found, "_hash_table.json") 

ft_flags = {type_enum: False for type_enum in TYPE_OSC} 
ft_flags[TYPE_OSC.COMTRADE_CFG_DAT] = True
ft_flags[TYPE_OSC.ARCH_ZIP] = True
ft_flags[TYPE_OSC.ARCH_7Z] = True 
ft_flags[TYPE_OSC.ARCH_RAR] = True 

copied_count_wf = finder_wf.copy_new_oscillograms(
    source_dir=initial_source_dir,
    dest_dir=staging_dir_found,
    preserve_dir_structure=True,
    use_hashes=True,
    file_type_flags=ft_flags,
    max_archive_depth=1 
)
print(f"\nЭтап 1: Скопировано {copied_count_wf} осциллограмм в {staging_dir_found}")
print("Содержимое staging_dir_found:")
for root, dirs, files in os.walk(staging_dir_found):
    level = root.replace(staging_dir_found, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f_name in files: print(f"{sub_indent}{f_name}")

## Этап 2: Анонимизация (`DataAnonymizer`)
Анонимизируем файлы в `staging_dir_found` и сохраняем их в `staging_dir_anonymized`.
DataAnonymizer сам переименовывает файлы в хеши и кладет их в ту же директорию, где нашел.
Поэтому для чистоты эксперимента, сначала скопируем файлы в `staging_dir_anonymized`, а потом анонимизируем их там.

In [ ]:
if os.path.exists(staging_dir_anonymized): shutil.rmtree(staging_dir_anonymized)
shutil.copytree(staging_dir_found, staging_dir_anonymized, dirs_exist_ok=True) 
print(f"Скопированы файлы из {staging_dir_found} в {staging_dir_anonymized} для анонимизации.")

anonymizer_wf = DataAnonymizer()
anonymizer_wf.anonymize_directory(staging_dir_anonymized) # Использует print для вывода ошибок/варнингов

print(f"\nЭтап 2: Анонимизация завершена в {staging_dir_anonymized}.")
print("Содержимое staging_dir_anonymized (файлы должны быть переименованы в хеши):")
for root, dirs, files in os.walk(staging_dir_anonymized):
    level = root.replace(staging_dir_anonymized, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f_name in files: print(f"{sub_indent}{f_name}")

## Этап 3: Каталогизация имен сигналов (`SignalNameManager`)
Создадим каталог имен сигналов из анонимизированных файлов.

In [ ]:
sm_wf = SignalNameManager()
catalog_path_wf = os.path.join(output_configs_dir, "workflow_signal_catalog.csv")
sm_wf.find_signal_names(staging_dir_anonymized, output_csv_path=catalog_path_wf, is_print_message=True)

if os.path.exists(catalog_path_wf):
    print("\nСодержимое каталога сигналов:")
    try: from IPython.display import display; display(pd.read_csv(catalog_path_wf))
    except ImportError: print(pd.read_csv(catalog_path_wf))

## Этап 4: Фильтрация по активности (`OscillogramActivityFilter`)
Отфильтруем "пустые" осциллограммы из анонимизированной директории. Результаты (список активных файлов) сохраним в CSV. Затем скопируем только активные файлы в `staging_dir_active`.

In [ ]:
activity_config_wf = {
    'channels_to_analyze_patterns': ['U_Active', 'I_Stable', 'U_Archived'], 
    'current_channel_id_patterns': ['i_'],
    'voltage_channel_id_patterns': ['u_'],
    'use_norm_osc': False, 
    'raw_signal_analysis': {
        'initial_window_check_periods': 1, 
        'h1_vs_hx_ratio_threshold_U': 2, 
        'h1_vs_hx_ratio_threshold_I': 2,
        'min_initial_h1_amplitude_for_rel_norm': 0.005, 
        'thresholds_raw_current_relative': {'delta': 0.1, 'std_dev': 0.05, 'max_abs': 0.1},
        'thresholds_raw_voltage_relative': {'delta': 0.1, 'std_dev': 0.05, 'max_abs': 0.1}
    },
    'verbose': False
}
activity_filter_wf = OscillogramActivityFilter(config=activity_config_wf, normalizer=None)
active_files_report_path = os.path.join(output_configs_dir, "workflow_active_files.csv")

activity_filter_wf.verbose = True
activity_filter_wf.filter_directory(staging_dir_anonymized, active_files_report_path)
activity_filter_wf.verbose = False

if os.path.exists(active_files_report_path):
    active_files_df = pd.read_csv(active_files_report_path)
    if not active_files_df.empty and 'active_files' in active_files_df.columns:
        print(f"\nКопирование {len(active_files_df)} активных файлов в {staging_dir_active}...")
        for _, row in active_files_df.iterrows():
            active_cfg_name = row['active_files'] 
            for root, _, files in os.walk(staging_dir_anonymized):
                if active_cfg_name in files:
                    src_cfg_path = os.path.join(root, active_cfg_name)
                    src_dat_path = os.path.join(root, active_cfg_name[:-4] + ".dat")
                    
                    relative_dir = os.path.relpath(root, staging_dir_anonymized)
                    dest_subdir_active = os.path.join(staging_dir_active, relative_dir)
                    os.makedirs(dest_subdir_active, exist_ok=True)
                    
                    dest_cfg_path = os.path.join(dest_subdir_active, active_cfg_name)
                    dest_dat_path = os.path.join(dest_subdir_active, active_cfg_name[:-4] + ".dat")
                    
                    if os.path.exists(src_cfg_path): shutil.copy2(src_cfg_path, dest_cfg_path)
                    if os.path.exists(src_dat_path): shutil.copy2(src_dat_path, dest_dat_path)
                    break 
        print(f"Содержимое {staging_dir_active} после копирования активных файлов:")
        for root, dirs, files in os.walk(staging_dir_active):
            level = root.replace(staging_dir_active, '').count(os.sep)
            indent = ' ' * 4 * (level); print(f"{indent}{os.path.basename(root)}/")
            sub_indent = ' ' * 4 * (level + 1)
            for f_name in files: print(f"{sub_indent}{f_name}")
    else:
        print("Активных файлов для копирования не найдено.")
else:
    print("Отчет об активных файлах не создан.")

## Этап 5: Генерация коэффициентов нормализации (`NormalizationCoefficientGenerator`)
Используем активные, анонимизированные осциллограммы из `staging_dir_active` для генерации `norm.csv`.

In [ ]:
workflow_norm_csv_path = os.path.join(staging_dir_active, "norm.csv") 

coeff_gen_wf = NormalizationCoefficientGenerator(
    osc_path=staging_dir_active, 
    prev_norm_csv_path="", 
    bus=1 
)
# coeff_gen_wf.is_print_message = True # If needed

print("\nЗапуск генерации коэффициентов нормализации...")
coeff_gen_wf.normalization()

if os.path.exists(workflow_norm_csv_path):
    print(f"\nФайл коэффициентов '{workflow_norm_csv_path}' успешно создан.")
    try: from IPython.display import display; display(pd.read_csv(workflow_norm_csv_path).head())
    except ImportError: print(pd.read_csv(workflow_norm_csv_path).head())
else:
    print(f"\nОшибка: Файл '{workflow_norm_csv_path}' не был создан.")

## Этап 6: Преобразование в CSV (`OscillogramToCsvConverter`)
Конвертируем активные, анонимизированные осциллограммы в CSV, используя сгенерированные коэффициенты нормализации.

In [ ]:
wf_analog_names_path = os.path.join(output_configs_dir, "wf_dict_analog.json")
wf_analog_content = { "bus1": { 
    "U_Active": ["U_Active"], "I_Stable": ["I_Stable"], 
    "U_Active_Sub": ["U_Active_Sub"], "U_Archived": ["U_Archived"]
} }
with open(wf_analog_names_path, "w", encoding="utf-8") as f: json.dump(wf_analog_content, f)
wf_discrete_names_path = os.path.join(output_configs_dir, "wf_dict_discrete.json")
wf_discrete_content = { "bus1": { "ML_Event": ["ML_Event_Dummy"]}}
with open(wf_discrete_names_path, "w", encoding="utf-8") as f: json.dump(wf_discrete_content, f)

if not os.path.exists(workflow_norm_csv_path):
    print(f"Файл {workflow_norm_csv_path} не найден! Пропускаем этап конвертации в CSV.")
else:
    normalizer_workflow = OscillogramNormalizer(norm_coef_file_path=workflow_norm_csv_path, is_print_message=True)

    converter_wf = OscillogramToCsvConverter(
        normalizer=normalizer_workflow,
        raw_path=staging_dir_active, 
        csv_path=output_configs_dir, 
        uses_buses=['1'], 
        dict_analog_names_path=wf_analog_names_path,
        dict_discrete_names_path=wf_discrete_names_path,
        is_print_message=True
    )
    converter_wf.number_periods = 1 

    final_csv_name = "final_dataset.csv"
    print(f"\nЗапуск финального преобразования в CSV (выход: {final_csv_name})...")
    df_final = converter_wf.create_csv(csv_name=final_csv_name, is_cut_out_area=True, is_simple_csv=False)

    if df_final is not None and not df_final.empty:
        print(f"\nПервые строки итогового CSV ({final_csv_name}):")
        try: from IPython.display import display; display(df_final.head())
        except ImportError: print(df_final.head())
        print(f"Колонки: {df_final.columns.tolist()}")
    else:
        print(f"Итоговый CSV не был создан или пуст.")

## Этап 7: Специализированный анализ (Пример: `SPEFAnalyzer`)
Это опциональный шаг. Можно запустить любой из анализаторов на полученных данных.
Для `SPEFAnalyzer` потребовалась бы более специфичная настройка `norm.csv` и конфигурации самого анализатора.
Здесь просто обозначим возможность.

In [ ]:
print("\nПример вызова SPEFAnalyzer (закомментирован, требует своей конфигурации):")
# spef_config_wf = { 
# 'VALID_NOMINAL_VOLTAGES': {6000.0/np.sqrt(3)}, # Пример
# 'SPEF_THRESHOLD_U0': 0.05, 'SPEF_THRESHOLD_Un': 0.03, 
# 'SPEF_MIN_DURATION_PERIODS': 1, 
# 'SIMILAR_AMPLITUDES_FILTER_ENABLED': True, 
# 'SIMILAR_AMPLITUDES_MAX_RELATIVE_DIFFERENCE': 0.15, 
# 'verbose': True, 'norm_yes_phrase': 'YES' 
# }
# if os.path.exists(workflow_norm_csv_path):
#     norm_coeffs_df_wf = pd.read_csv(workflow_norm_csv_path)
#     spef_analyzer_wf = SPEFAnalyzer(
#         config=spef_config_wf,
#         normalizer=normalizer_workflow, # Уже есть
#         bus_splitter=converter_wf,    # Уже есть (OscillogramToCsvConverter)
#         norm_coef_df=norm_coeffs_df_wf
#     )
#     spef_report_path_wf = os.path.join(output_configs_dir, "workflow_spef_report.csv")
#     spef_error_log_wf = os.path.join(output_configs_dir, "workflow_spef_errors.log")
#     spef_analyzer_wf.analyze_directory(staging_dir_active, spef_report_path_wf, spef_error_log_wf)
#     if os.path.exists(spef_report_path_wf):
#         print("Отчет SPEF:")
#         try: from IPython.display import display; display(pd.read_csv(spef_report_path_wf))
#         except ImportError: print(pd.read_csv(spef_report_path_wf))
# else:
#     print("Файл norm.csv не найден, анализ SPEF пропущен.")

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(base_workflow_dir):
        shutil.rmtree(base_workflow_dir)
        print(f"\nВременная директория {base_workflow_dir} удалена.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {base_workflow_dir}: {e}")